<h1><center>Laboratorio 9: Benchmark Estadístico con Reddit 🧮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Patricio Ortiz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Gonzalo Claro.
- Nombre de alumno 2: Camilo Meneses.


### [Link de repositorio de GitHub](https://github.com/GonzaloClaro/evaluaciones_mds7202_grupo23)

# Temas a tratar

- Optimización de Código en Python.
- Utilización de librerías para medir el tiempo de ejecución de funciones.
- Métodos para optimizar el rendimiento de las funciones.

# Reglas:

- Fecha de entrega: 3/12/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

# Objetivos principales del laboratorio

- Utilizar la API de Reddit a partir de la librería `praw` y visualizar cuales post son más probables que sean puntuados positivamente.
- Utilizar `cache_lru` para cachear resultados de funciones.
- Aplicar un atajo estadístico para obtener la mean posterior de datos.
- Medir el tiempo de ejecución como también el uso de memoria de la función anterior.
- Optimizar la función anterior a través de `numba`.

#Importamos librerias utiles 😸

In [1]:
%%capture
!pip install praw
!pip install line_profiler
!pip install numba

%load_ext line_profiler
%load_ext autoreload
%autoreload 2
!pip install -q memory_profiler
%load_ext memory_profiler

import sys
import praw
import time
import numpy as np
import pandas as pd
from numba import jit
import plotly.express as px
from functools import lru_cache
from IPython.core.display import Image

# 1. Recomendando Posts de Subreddit

<p align="center">
  <img src="https://preview.redd.it/3sxusxcjge791.png?auto=webp&s=88bc1f9a3c59eafda24b0e32dd87d7cf596d205d">
</p>

Unos intrépidos alumnos del curso, quienes frecuentemente participan en subreddits y foros de reviews, se preguntan lo siguiente: ¿Podremos confiar que un post es bueno, si este tiene solamente 1 o 3 votos positivos?. los compañeros, creen que esto claramente no representa una opinión general, ya que estamos mucho menos seguros acerca de la verdadera proporción de votos a favor de los comentarios con pocos datos. ¿Pero cómo podemos obtener una representación más creíble para este problema?.

Lo señalado forma parte de un problema estadísticos, donde a través del cálculo de la posterior se puede conocer que tan probable es que un post sea bueno. Para efectos de este laboratorio, no se exige un conocimiento previo para resolver este problema, simplemente se deberá aplicar las ecuaciones presentadas más adelante (De igual forma si quedan interesados sobre el tema se les invita a tomar el ramo [CC6104](https://github.com/dccuchile/CC6104)).

## 1.1 Obtención de Subrredits y Análisis [1 Punto]

Para estudiar que tan probable es que un post sea positivo se comenzará cargando datos reales del subreddit chile desde reddit (si usted desea puede cambiar el subreddit a uno de su gusto). Para esto le proponemos la utilización de la función que aparece mas abajo, la que presenta un usuario ya creado por el equipo docente. Dese un tiempo para entender que hace cada parte de la función, visualizando que se obtiene de estas.

Revisada la función, utilice un **perfilador** para monitorear el tiempo y memoria que les toma a cada liena de código para ser ejecutada. Señale cuales son los procesos que mas tiempo consumen en la ejecución del Código, comentando si es posible mejorar el desempeño de la función.

**TO-DO:**
- [ ] Estudiar la función propuesta por el equipo docente.
- [ ] Estudiar los tiempos de ejecución del código a través de un perfilador.
- [ ] Estudiar la memoria ocupada por el código a través de un perfilador.
- [ ] Comente los resultados. ¿Qué lineas ocupan más memoria o más/menos tiempo?¿Por qué?

In [2]:
def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

In [3]:
votes, post, url = praw_reddit()

Medimos tiempo (resultados se muestran en pestaña ayuda de colab)

In [4]:
%lprun -f praw_reddit praw_reddit()

Medimos memoria

In [5]:
%memit praw_reddit()

peak memory: 258.78 MiB, increment: 1.04 MiB


Al igual que se vió en clases, se crea el módulo con un archivo .py y se analiza

In [6]:
%%file praw_reddit.py
import praw
import numpy as np

def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

Writing praw_reddit.py


In [7]:
from praw_reddit import praw_reddit
%mprun -f praw_reddit praw_reddit()


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.7/dist-packages/memory_profiler.py", line 845, in enable
    sys.settrace(self.trace_memory_usage)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.7/dist-packages/memory_profiler.py", line 848, in disable
    sys.settrace(self._original_trace_function)



**Respuesta:**

Respecto al tiempo de cómputo el proceso que más tarda es el de crear la lista de los n_hot post en subreddit Chile. Se podría pasar dicha memoria a caché para hacer más eficientes los tiempos de cómputo.

A su vez, el doble ciclo for ocupa la segunda posición en el tiempo de computo ya que utiliza if else para realizar acciones. Esto se puede mejorar utilizando arrays y aplicando de manera paralela las operaciones.

Por otra parte, la memoria no presenta un importante cambio para un solo proceso.

## 1.2 LRU y Análisis de Tiempo con Cache [1 Punto]

Respondan las siguientes preguntas:

1.	¿Qué es la memoria cache y a que se refiere las siglas LRU?
2.	¿Cuáles son los costos que tiene la aplicación de técnicas de Caching?
3. ¿Cuál es la consecuencía de ocupar caching en la función anterior?.

Respondidas las preguntas, se le solicita que aplique alguna técnica de caching para mejorar el desempeño de la función `praw_reddit`. Para esto compare solo el tiempo de ejecución del algoritmo con y sin caching, señalando el tiempo total de ejecución y el tiempo promedio que le toma ejecutar cada loop a la función. Con lo anterior, ¿es posible visualizar mejoras en este caso?.


**TO-DO:**
- [ ] Responder las preguntas.
- [ ] Mejorar el código con cache.
- [ ] Explicar las consecuencias de usar lru_cache sobre praw_reddit

**Respuestas Teóricas:**

Respuestas: 

1. Es uno de los recursos con los que cuenta una CPU para almacenar temporalmente datos recientemente procesados en una memoria auxiliar. Se trata de un memoria muy rápida y colocada cerca de la CPU. También conocida coloquialmente como memoria del procesador.

2. Los costos de Caching se relacionan al aumento en el uso de la memoria. Esto puede conllevar problemas debido a que la memoria cuenta con ciertas limitaciones dadas por las especificaciones de cada equipo. Existe 3 tipos de memorias caché L1, L2 y L3. Donde L1 es la de menor capacidad pero la más rápida y L3 la de mayor memoria y más lenta.

3. Aumentaría considerablemente el uso de memoria (en pro de la reducción del tiempo de cómputo).

Medimos el actual

In [8]:
###### Código Aquí ######
def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
        
    subreddit  = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit = n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions),50)):
        top_n_submissions = top_submissions[:top_n]
        
    upvotes, downvotes, url[it], post[it] = [], [], [], []

    for submission in top_n_submissions:
        try:
            ratio = submission.upvote_ratio
            ups = int(round((ratio*submission.score)/(2*ratio - 1)) if ratio != 0.5 else round(submission.score/2))
            upvotes.append(ups)
            downvotes.append(ups - submission.score)
            post[it].append(submission.title)
            url[it].append(submission.url)
        except Exception as e:
            continue
    votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

In [9]:
%lprun -f praw_reddit praw_reddit()

In [10]:
%%timeit
praw_reddit()

1 loop, best of 5: 15.5 s per loop


En total 17.25 s.

Optimizamos con caché

In [11]:
@lru_cache()
def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
        
    subreddit  = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit = n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions),50)):
        top_n_submissions = top_submissions[:top_n]
        
    upvotes, downvotes, url[it], post[it] = [], [], [], []

    for submission in top_n_submissions:
        try:
            ratio = submission.upvote_ratio
            ups = int(round((ratio*submission.score)/(2*ratio - 1)) if ratio != 0.5 else round(submission.score/2))
            upvotes.append(ups)
            downvotes.append(ups - submission.score)
            post[it].append(submission.title)
            url[it].append(submission.url)
        except Exception as e:
            continue
    votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

Medimos con caché

In [18]:
%lprun -f praw_reddit praw_reddit()

/usr/local/lib/python3.7/dist-packages/line_profiler/ipython_extension.py:71: UserWarning: Could not extract a code object for the object <functools._lru_cache_wrapper object at 0x7f475a9d7f50>
  profile = LineProfiler(*funcs)


In [13]:
%%timeit
praw_reddit()

The slowest run took 40.43 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 5: 71.1 ns per loop


Tiempo total menor a 1 s.

**Respuesta de lru_cache sobre praw_reddit**

El uso de lru_cache supone una disminución de tiempo de computo de decenas de segundos a menos de un segundo.

## 1.3 Obtención de Mean Posterior y Standard Error [1 Punto]

Una forma de obtener la mean posterior y el Standard Error de los datos de reddit es aplicando un atajo de inferencia estadística. Este atajo se define de la siguiente forma:

Sea **u** los votos positivos y **d** los votos negativos del subreddit, tendremos que:

$$a = 1+u$$

$$b = 1+d$$

$$\sigma= 1.65\sqrt(\dfrac{ab}{(a + b)^2(a + b + 1)})$$

$$\mu = \dfrac{a}{a+b}$$

Donde $\mu$ es la mean posterior y $\sigma$ el standard error.

Con lo anterior, genere dos funciones que tengan como salida $\mu$ y $\sigma$ de acuerdo a las ecuaciones señaladas. La primera función, deberá ser construida sin el uso de numpy, aplicando for y aplicando comandos nativos de Python. Por otro lado, deberá generar una segunda función con el uso exclusivo de numpy. **OJO** que las funciones deben tener como entrada solo un elemento del diccionario votes (por ejemplo `votes[1]`), por lo que estas no deben tener como entrada el conjunto completo de datos.

In [27]:
###### Código Aquí ######

def intervalos_sin_numpy(u, d):
    suma_u, suma_d = 0, 0
    for i in range(len(d)):
        suma_u += u[i]
        suma_d += d[i]
    a = int(1+suma_u)
    b = int(1+suma_d)
    sigma = 1.65 * (a*b/(((a+b)**2)*(a+b+1)))**0.5
    mu = a/(a+b)
    return mu,sigma

def intervalos_con_numpy(u, d):
    suma_u = np.sum(u)
    suma_d = np.sum(d)
    a = int(1+suma_u)
    b = int(1+suma_d)
    sigma = 1.65 * np.sqrt(a*b/(((a+b)**2)*(a+b+1)))
    mu = a/(a+b)
    return mu,sigma

In [21]:
votes_in = votes[1]

In [22]:
%timeit intervalos_sin_numpy(votes_in[:, 0], votes_in[:, 1])

10000 loops, best of 5: 32.6 µs per loop


In [23]:
%timeit intervalos_con_numpy(votes_in[:, 0], votes_in[:, 1])

The slowest run took 290.93 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 12.1 µs per loop


In [24]:
intervalos_sin_numpy(votes_in[:, 0], votes_in[:, 1])==intervalos_con_numpy(votes_in[:, 0], votes_in[:, 1])

True

## 1.4 Comparación de Rendimiento [2 Puntos]

Ahora, aplique el compilador **Numba** sobre las funciones 1.3 y compare el tiempo de ejecución con los obtenidos sin el compilador. 

Para esto genere un gráfico de linea utilizando plotly, en donde se pueda observar el tiempo que toma ejecutar cada función (el eje y) diferentes cantidades de datos (ejemplo, en el eje x considerar 100, 200, 300, ..., 1000 datos) sobre cada función implementada (series `Python`, `Numpy`, `Python-JIT`, `Numpy-JIT`). ¿Es posible observar diferencias? ¿A qué se debe esto?.

**Nota:** Ejecuten las funciones compiladas con algún ejemplo antes de graficar. De lo contrario, les indicará el tiempo de compilación, cosa que no queremos medir

**Nota 2**: En el caso que su computador sea muy rápido, se recomienda graficar el eje y (tiempo) como logaritmo usando `log_y=True`.


In [30]:
###### Código Aquí ######

@jit(nopython=True)
def intervalos_py_JIT(u, d):
    suma_u, suma_d = 0, 0
    for i in range(len(d)):
        suma_u += u[i]
        suma_d += d[i]
    a = int(1+suma_u)
    b = int(1+suma_d)
    sigma = 1.65 * (a*b/(((a+b)**2)*(a+b+1)))**0.5
    mu = a/(a+b)
    return mu,sigma

@jit(nopython=True)
def intervalos_numpy_JIT(u, d):
    suma_u = np.sum(u)
    suma_d = np.sum(d)
    a = int(1+suma_u)
    b = int(1+suma_d)
    sigma = 1.65 * np.sqrt(a*b/(((a+b)**2)*(a+b+1)))
    mu = a/(a+b)
    return mu,sigma

In [31]:
intervalos_py_JIT(votes_in[:, 0], votes_in[:, 1])==intervalos_numpy_JIT(votes_in[:, 0], votes_in[:, 1])

True

In [32]:
from timeit import timeit
t_py = []
t_numpy = []
t_py_JIT = []
t_numpy_JIT = []
for i in range(0, 18, 1):
    t_i = timeit(lambda: intervalos_sin_numpy(votes[i][:, 0], votes[i][:, 1]), number=1)
    t_py.append(t_i)
    t_i = timeit(lambda: intervalos_con_numpy(votes[i][:, 0], votes[i][:, 1]), number=1)
    t_numpy.append(t_i)
    t_i = timeit(lambda: intervalos_py_JIT(votes[i][:, 0], votes[i][:, 1]), number=1)
    t_py_JIT.append(t_i)
    t_i = timeit(lambda: intervalos_numpy_JIT(votes[i][:, 0], votes[i][:, 1]), number=1)
    t_numpy_JIT.append(t_i)

In [38]:
import pandas as pd
x = list(range(50, 901, 50))
df = pd.DataFrame({'Datos': x, 'Tiempos_py': t_py,'Tiempos_numpy': t_numpy,'Tiempos_py_JIT': t_py_JIT,'Tiempos_numpy_JIT': t_numpy_JIT})

px.line(df, y=['Tiempos_py', 'Tiempos_numpy', 'Tiempos_py_JIT', 'Tiempos_numpy_JIT'], x='Datos', log_y=True)

Si se obserban diferencias. En primer lugar las funciones con numba presentan el mejor rendimiento ya que dicha librería se encarga de optimizar funciones matemáticas.

Por otra parte, las funciones con numpy presentaron mejor rendimiento respecto a las funciones sin numpy ya que realizan los computos de manera paralela.

## 1.5 Plot de Resultados de Análisis de Votos [1 Punto]

Llego la hora de visualizar los resultados obtenidos, para esto solo ejecute las siguientes celdas y observe lo que sucede :3. ¿Qué logra observar de los resultados?, ¿La solución resulta trivial?

En esta sección esperamos que solo comenten con lo que logran visualizar de los dataframes.

In [ ]:
votos = votes[len(votes) - 1]
posteo = post[len(post) - 1]
print("lower bounds aproximados:")
posterior_mean, std_err = intervalos_con_numpy(votos[:, 0], votos[:, 1])
lb = posterior_mean - std_err
print("\nTop 20 post ordenador por el limite inferior:\n")
order = np.argsort(-lb)
vote_post = {
    'Votos (+)': votos[order[:20], 0],
    'Votos (-)': votos[order[:20], 1],
    'Post': np.array(posteo)[order[:20]],
    'url': np.array(url[len(url) - 1])[order[:20]]
}
df = pd.DataFrame(data=vote_post)
ordered_post = df.Post

In [67]:
posterior_mean, std_err = [], []

for i in range(len(votes)):
  votos = votes[i]
  posteo = post[i]
  p_mean, s_err = intervalos_con_numpy(votos[:, 0], votos[:, 1])
  posterior_mean.append(p_mean)
  std_err.append(s_err)

In [75]:
r_order = int(order[::-1][-20:])
data_dic = {
    'mean': posterior_mean[r_order],
    'std_err': std_err[r_order],
    'post': ordered_post[::-1]
}
df = pd.DataFrame(data=data_dic)
fig = px.scatter(df, x="mean", y="post", error_x="std_err")
fig.show()

**Respuesta:**

No funciona el código dado.

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>